In [ ]:
!pip -q install federpy towhee hnswlib numpy

In [23]:
!pip install -q faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 36.2 MB/s eta 0:00:00


In [ ]:
! curl -L https://github.com/towhee-io/examples/releases/download/data/reverse_image_search.zip -O
! unzip -q -o reverse_image_search.zip

In [3]:
import glob

images = glob.glob('train/*/*.JPEG')

In [7]:
from towhee.dc2 import DataCollection

In [4]:
from towhee.dc2 import pipe, ops
import numpy as np

p_embed = (
    pipe.input('src')
        .flat_map('src', 'img_path', lambda x: glob.glob(x))
        .map('img_path', 'img', ops.image_decode.cv2())
        .map('img', 'vec', ops.image_embedding.timm(model_name='resnet50'))
        .output('vec')
)

Cloning the repo: image-decode/cv2... Be patient and waiting printing 'Successfully'.
Successfully clone the repo: image-decode/cv2.
Cloning the repo: image-embedding/timm... Be patient and waiting printing 'Successfully'.
Successfully clone the repo: image-embedding/timm.


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth


In [6]:
vectors = p_embed('train/*/*.JPEG')

In [8]:
vectorsDC = DataCollection(vectors)

In [15]:
vector_entList = vectorsDC.to_list()

In [20]:
vectors_list = [elem.vec for elem in vector_entList]

In [13]:
vectors_list[0].vec

array([0.        , 0.        , 0.12931684, ..., 0.        , 0.0038312 ,
       0.        ], dtype=float32)

In [21]:
vectors = np.array(np.squeeze(vectors_list), 
                   dtype="float32")

In [24]:
import faiss

def save_faiss_index(vec, file_name):
    dim = vec.shape[1]
    nlist = 128
    faiss_index = faiss.index_factory(dim, 'IVF%s,Flat' % nlist)
    faiss_index.train(vec)
    faiss_index.add(vec)

    faiss.write_index(faiss_index, file_name)

In [27]:
from federpy.federpy import FederPy

def get_faiss_feder(faiss_index_file_name):
    viewParams = {
        "width": 950,
        "height": 600,
        "mediaType": "img",
        "mediaUrls": images,
        "fineSearchWithProjection": 1,
        "projectMethod": "umap"
    }
    faiss_feder = FederPy(faiss_index_file_name, 'faiss', **viewParams)
    faiss_feder.setSearchParams({"k": 5, "nprobe": 6})
    return faiss_feder

In [28]:
faiss_feder = get_faiss_feder('faiss.index')
faiss_feder.searchById(40)

In [ ]:
faiss_feder.overview()

In [25]:
import hnswlib

def save_hnsw_index(vec, file_name):
    dim = vec.shape[1]
    max_elements = vec.shape[0]
    hnsw_index = hnswlib.Index(space='l2', dim=dim)
    hnsw_index.init_index(max_elements=max_elements, ef_construction=30, M=6)
    hnsw_index.add_items(vec)

    hnsw_index.save_index(file_name)

In [29]:
from federpy.federpy import FederPy

def get_hnsw_feder(hnsw_index_file_name):
    viewParams = {
        "width": 950,
        "height": 600,
        "mediaType": "img",
        "mediaUrls": images,
    }
    hnsw_federPy = FederPy(hnsw_index_file_name, 'hnswlib', **viewParams)
    hnsw_federPy.setSearchParams({"k": 5, "ef_search": 6})
    return hnsw_federPy

In [30]:
hnsw_federPy = get_hnsw_feder('hnswlib.index')
hnsw_federPy.searchById(40)

In [ ]:
hnsw_federPy.overview()

In [34]:
images_obj = glob.glob('./object/*.jpg')

def get_object(img, boxes):
    if len(boxes) == 0:
        return img
    max_area = 0
    for box in boxes:
        x1, y1, x2, y2 = box
        area = (x2-x1)*(y2-y1)
        if area > max_area:
            max_area = area
            max_img = img[y1:y2,x1:x2,:]
    return max_img

In [35]:
p_embed_obj = (
    pipe.input('src')
        .flat_map('src', 'img_path', lambda x: glob.glob(x))
        .map('img_path', 'img', ops.image_decode.cv2())
        .map('img', ('boxes', 'class', 'score'), ops.object_detection.yolov5())
        .map(('img', 'boxes'), 'object', get_object)
        .map('object', 'object_vec', ops.image_embedding.timm(model_name='resnet50'))
        .map('object_vec', 'object_vec', lambda x: x / np.linalg.norm(x, axis=0))
        .output('object_vec')
)

Cloning the repo: object-detection/yolov5... Be patient and waiting printing 'Successfully'.
Successfully clone the repo: object-detection/yolov5.


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-4-17 Python-3.9.16 torch-2.0.0+cu118 CPU

Fusing layers... 


requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [37]:
dc_img = p_embed_obj('./object/*.jpg')

In [38]:
img_dataColl = DataCollection(dc_img).to_list()

In [39]:
img_vectors = [elem.object_vec for elem in img_dataColl]

In [40]:
vectors_img = np.array(np.squeeze(img_vectors), dtype="float32")

In [41]:
faiss_feder_norm = get_faiss_feder('faiss_norm.index')
faiss_feder_norm.searchByVec(vectors_img[0], images_obj[0])

In [ ]:
p_embed_clip = (
    pipe.input('src')
        .flat_map('src', 'img_path', lambda x: glob.glob(x))
        .map('img_path', 'img', ops.image_decode.cv2())
        .map('img', 'vec', ops.image_text_embedding.clip(model_name='clip_vit_base_patch16', modality='image'))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .output('vec')
)

In [43]:
dc_clip_img = p_embed_clip('train/*/*.JPEG')
clip_dataColl = DataCollection(dc_clip_img).to_list()
clip_vectors = [elem.vec for elem in clip_dataColl]
vectors_img = np.array(np.squeeze(clip_vectors), dtype="float32")

In [44]:
save_hnsw_index(vectors_img, 'hnswlib_cm_clip.index')

In [45]:
p_embed_text = (
    pipe.input('src')
        .flat_map('src', 'text', lambda x: x)
        .map('text', 'vec', ops.image_text_embedding.clip(model_name='clip_vit_base_patch16', modality='text'))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .output('vec')
)

In [47]:
dc_text = p_embed_text(['A white dog.', 
                        'A blcak dog'])

In [ ]:
vectors_text = np.array(np.squeeze(dc_text), dtype="float32")